# Tutorial about rendering LocData

In [ ]:
from pathlib import Path

%matplotlib inline
# %matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import locan as lc

In [ ]:
lc.show_versions(system=False, dependencies=False, verbose=False)

## Synthetic data

Localizations are simulated that are distributed according to a Neyman-Scott distribution (blobs).

In [ ]:
rng = np.random.default_rng(seed=1)

In [ ]:
locdata = lc.simulate_Thomas(parent_intensity=1e-5, region=((0, 1000), (0, 1000)), cluster_mu=100, cluster_std=10, seed=rng)

locdata.print_summary()

Since localization data is kept as a pandas dataframe standard plotting routines from pandas or matplotlip can be used.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

## Render by simple 2D binning

A method for simply binning localization data in 2D pixels is provided.

In [ ]:
img, bins, label = lc.histogram(locdata, bin_size=10)

The intensity values of the binned locdata are distributed as:

In [ ]:
plt.hist(img.ravel(), bins=256, range=(1, 50), fc='k', ec='k');

The intensity values can be rescaled in many ways. There are normalization classes and a convenience function in the locan.render.transformation module with predefined transformations as listed in locan.Trafo:

In [ ]:
list(lc.Trafo)

In [ ]:
img_new = lc.adjust_contrast(img, rescale=lc.Trafo.STANDARDIZE)
epsilon = np.finfo(float).resolution
plt.hist(img_new.ravel(), bins=256, range=(epsilon, 1), fc='k', ec='k');

In [ ]:
norm = lc.HistogramEqualization(power=1, mask=img>0)
img_new = norm(img)
plt.hist(img_new.ravel(), bins=256, range=(epsilon, 1), fc='k', ec='k');

The `render_2d` method can directly provide a new figure as output.

In [ ]:
lc.render_2d(locdata, bin_size=10);

Or it can be used within the matplotlib environment.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
lc.render_2d(locdata, ax = ax, bin_size=10, cmap='viridis')
plt.show()

Intensity is per default scaled to the min and max intensity values but can be rescaled by applying any norm function as described for matplotlib.imshow:

In [ ]:
norm = plt.Normalize(vmax=10)
lc.render_2d(locdata, bin_size=10, norm=norm);

Intensity can also be rescaled by normalization functions as defined in locan.Trafo. Histogram equlization yields this image:

In [ ]:
lc.render_2d(locdata, bin_size=10, rescale=lc.Trafo.EQUALIZE);

Histogram equlization with a power-intensification of p=0.3 yields this image:

In [ ]:
lc.render_2d(locdata, bin_size=10, rescale=lc.Trafo.EQUALIZE_0P3);

Any callable normalization object can be passed into the rescale kwarg:

In [ ]:
norm = lc.HistogramEqualization(power=0.3)
lc.render_2d(locdata, bin_size=10, rescale=norm);

The image size is set automatically to the min and max coordinates but can be set to (0, max) or an arbitrary range.

In [ ]:
lc.render_2d(locdata, bin_size=10, bin_range='zero');

In [ ]:
lc.render_2d(locdata, bin_size=10, bin_range=((0, 300),(200, 500)));

## Use different libraries for rendering

Rendering can also be carried out with a different render engine. Choose one of the following (MPL is the standard matplotlib):

In [ ]:
list(lc.RenderEngine)

### napari

As external viewer you can use **napari**.

For napari the qt gui interface has to be activated. Make sure to run the following command manually in a single cell and give enough time for this procedure to complete in the background.

or alternatively:

### mpl-scatter-density

For interactive viewing with variable binning, **mpl-scatter-density** is helpful. To run this the notebook has to be run with a new kernel applying the magic command `%matplotlib widget`  before matplotib is imported.

In [ ]:
lc.render_2d(locdata, render_engine=lc.RenderEngine.MPL_SCATTER_DENSITY);

## Choose colormaps

Colormaps can be chosen in matplotlib, napari and other visualization tools.

Colormaps are identified through specific class instances or by a name.

locan.Colormap serves as adapter class for the various visualization tools.

A Colormap instance can be created with the `get_colormap` function:

In [ ]:
colormap = lc.get_colormap("viridis")
colormap.name

In [ ]:
colormap.matplotlib

A mapping of names on Colormap instances is provided and can be extended by users. If a colormap name is provided to any rendering function, first the colormap_registry is searched, then the maplotlib registry and finally napari colormap names.

In [ ]:
lc.colormap_registry

Default colormaps are defined for use with locan and can be accessed through a mapping or an enum:

In [ ]:
lc.COLORMAP_DEFAULTS

In [ ]:
list(lc.Colormaps)

In [ ]:
lc.render_2d(locdata, bin_size=10, cmap=lc.Colormaps.CONTINUOUS_GRAY_REVERSE);